In [1]:
from collections import deque

def simulate_bytes(byte_positions, grid_size=7, num_bytes=1024):
    grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]
    length = len(byte_positions)
    for i in range(num_bytes):
        x, y = byte_positions[i % length]
        grid[y][x] = '#'
    return grid

def find_shortest_path(grid, start, goal):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    queue = deque([(start[0], start[1], 0)])  # (x, y, steps)
    visited = set()
    visited.add(start)

    while queue:
        x, y, steps = queue.popleft()
        if (x, y) == goal:
            return steps
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid) and (nx, ny) not in visited and grid[ny][nx] == '.':
                visited.add((nx, ny))
                queue.append((nx, ny, steps + 1))

    return -1  # if no path is found

# Load byte positions from a file
def load_byte_positions(filename):
    with open(filename, 'r') as file:
        positions = []
        for line in file:
            x, y = map(int, line.strip().split(','))
            positions.append((x, y))
        return positions

# Main execution logic
if __name__ == "__main__":
    byte_positions = load_byte_positions('input.txt')
    grid = simulate_bytes(byte_positions)
    start_position = (0, 0)
    end_position = (6, 6)
    result = find_shortest_path(grid, start_position, end_position)
    print(f"Minimum number of steps needed to reach the exit: {result}")


IndexError: list index out of range

In [ ]:
import heapq

def find_first_blocking_byte(input_file):
    # Read coordinates of falling bytes
    with open(input_file, 'r') as f:
        coordinates = [tuple(map(int, line.strip().split(','))) for line in f]
    
    # Define grid size and movement directions
    GRID_SIZE = 70
    DIRECTIONS = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    # Check if a path exists from start to goal
    def is_path_exists(corrupted):
        start = (0, 0)
        goal = (GRID_SIZE, GRID_SIZE)
        
        visited = set()
        queue = [start]
        visited.add(start)
        
        while queue:
            current = queue.pop(0)
            
            if current == goal:
                return True
            
            for dx, dy in DIRECTIONS:
                neighbor = (current[0] + dx, current[1] + dy)
                
                # Check if neighbor is valid (within grid and not corrupted)
                if (0 <= neighbor[0] <= GRID_SIZE and 
                    0 <= neighbor[1] <= GRID_SIZE and 
                    neighbor not in corrupted and 
                    neighbor not in visited):
                    queue.append(neighbor)
                    visited.add(neighbor)
        
        return False
    
    # Track corrupted spaces and find first blocking byte
    corrupted = set()
    for i, coord in enumerate(coordinates):
        corrupted.add(coord)
        
        # Check if this coordinate blocks the path
        if not is_path_exists(corrupted):
            return coord
    
    return None  # No blocking byte found

# Read input from file and solve
result = find_first_blocking_byte('input.txt')
print(f"First blocking byte coordinates: {result[0]},{result[1]}") 

In [1]:
import heapq

def parse_input(file_path):
    """Reads the input file and returns a list of tuples representing corrupted coordinates."""
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return [tuple(map(int, line.strip().split(','))) for line in lines]

def simulate_memory_corruption(corrupted_coords, grid_size):
    """Simulates the corruption on the grid."""
    grid = [[0 for _ in range(grid_size)] for _ in range(grid_size)]
    return grid

def is_valid(x, y, grid):
    """Checks if the coordinate (x, y) is valid and not corrupted."""
    return 0 <= x < len(grid) and 0 <= y < len(grid) and grid[y][x] == 0

def shortest_path(grid):
    """Finds the shortest path from top-left to bottom-right using A* algorithm."""
    start = (0, 0)
    end = (len(grid) - 1, len(grid) - 1)
    
    if grid[start[1]][start[0]] == 1 or grid[end[1]][end[0]] == 1:
        return -1  # No path if start or end is corrupted

    # Priority queue: (cost, x, y)
    pq = [(0, start[0], start[1])]
    visited = set()
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Left, Right, Up, Down

    while pq:
        cost, x, y = heapq.heappop(pq)
        if (x, y) in visited:
            continue
        visited.add((x, y))

        # Check if we reached the end
        if (x, y) == end:
            return cost

        # Explore neighbors
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny, grid) and (nx, ny) not in visited:
                heapq.heappush(pq, (cost + 1, nx, ny))

    return -1  # No path found

def find_blocking_byte(corrupted_coords, grid_size):
    """Finds the first byte that prevents the exit from being reachable."""
    grid = simulate_memory_corruption([], grid_size)

    for idx, (x, y) in enumerate(corrupted_coords):
        grid[y][x] = 1  # Add corruption
        if shortest_path(grid) == -1:
            return x, y  # Return the first blocking byte

def main():
    file_path = "input.txt"
    grid_size = 71  # Memory space dimensions (0 to 70 inclusive)
    corrupted_coords = parse_input(file_path)

    # Find the first blocking byte
    blocking_byte = find_blocking_byte(corrupted_coords, grid_size)
    print(f"The coordinates of the first byte that prevents the exit are: {blocking_byte[0]},{blocking_byte[1]}")

if __name__ == "__main__":
    main()

The coordinates of the first byte that prevents the exit are: 64,54
